In [ ]:
!pip install Faker

In [ ]:
def generate_product_name():
  brand = random.choice(['WayneTech', 'GothamCorp', 'CapsuleCorp', 'KaiTech', 'SuperSaiyan', 'Nimbus', 'Shenron', 'Batgear', 'Arkham', 'JokerTech', 'Batmobile', 'GokuMart', 'PlanetVegeta'])
  product = random.choice(['Smartphone', 'Laptop', 'Headphones', 'Smartwatch', 'Drone', 'VR Headset', 'Powerbank', 'SmartGlasses', 'Console', 'Keyboard'])
  model = f"{random.choice(['X', 'Pro', 'Max', 'Air'])} {random.randint(1,15)}"
  product_name = f"{product} {model}"
  return [brand, product_name]

In [ ]:
import csv
import json
import random
from faker import Faker

fake = Faker()

cities = []

try:
    with open('irish_cities.csv', mode='r', encoding='utf-8') as file_data:
      data = csv.DictReader(file_data)
      for row in data:
          if row['lat'] and row['lng']:
             population = "5000" if row['population'] == '' else row['population']
             city_data = ({
                 'name': row['city'],
                 'lat': float(row['lat']),
                 'lng': float(row['lng']),
                 'population': int(float(population)),
             })

             cities.append(city_data)

except FileNotFoundError:
    print("Error: irish_cities.csv file not found")


city_populations = [city['population'] for city in cities]

num_products = 100000
products = []

with open('products.ndjson', 'w') as f:
  for i in range(num_products):
      city = random.choices(cities, weights=city_populations, k=1)[0]

      lat_new = random.uniform(-0.05, 0.05)
      lng_new = random.uniform(-0.05, 0.05)

      data = generate_product_name()

      product = {
            "brand": data[0].lower(),
            "name": data[1].lower(),
            "price": round(random.uniform(5.0, 500.0), 2),
            "location": {
                "lon": city['lng'] + lat_new,
                "lat": city['lat'] + lng_new
            },
            "city": city['name'].lower()
        }

      f.write(json.dumps(product) + '\n')
      if (i + 1) % 10000 == 0:
          print(f"  ... {i + 1} / {num_products} generated")

print("Done. File 'products.ndjson' has been created.")


In [ ]:
import json

input_filename = 'products.ndjson'
output_filename = 'products_bulk.ndjson'

index_name = 'products'

metadata = {
    "index": {
        "_index": index_name
    }
}

action_line = json.dumps(metadata)

lines_processed = 0
with open(input_filename, 'r') as infile, open(output_filename, 'w') as outfile:
    for line in infile:
        if line.strip():
            outfile.write(action_line + '\n')
            outfile.write(line)
            lines_processed += 1

print(f"\nDone. Processed {lines_processed} documents.")


Done. Processed 100000 documents.
